<a href="https://colab.research.google.com/github/lstrgar/nma-cn-project-timelyjags/blob/main/video-segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from glob import glob

In [2]:
dropbox_link = 'https://www.dropbox.com/s/agxyxntrbwko7t1/participants_data.zip?dl=0'

# Use the dropbox link to download the data
os.environ["download_link"] = dropbox_link
!echo $download_link 
!wget -O participants_data.zip -c $download_link  
!wget -O example.nii -c https://github.com/Neural-Dynamics-of-Visual-Cognition-FUB/Algonauts2021_devkit/raw/main/example.nii

https://www.dropbox.com/s/agxyxntrbwko7t1/participants_data.zip?dl=0
--2021-07-09 15:13:47--  https://www.dropbox.com/s/agxyxntrbwko7t1/participants_data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/agxyxntrbwko7t1/participants_data.zip [following]
--2021-07-09 15:13:47--  https://www.dropbox.com/s/raw/agxyxntrbwko7t1/participants_data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6f05d05a2fd6ac4910e0020da6.dl.dropboxusercontent.com/cd/0/inline/BR_Rdjt_giZO7SLdxjtXvkQ1vqVjVEn1RNNE4FpyviAz5ny_ahm6Yq9Uh5lWhsidCX8O4fVmHAi0sz_veFFVhbreGmPp4--T7xl5zk0sbIOkbFCTPAJ2wo5HWe09vrEUQzf7OOoB_8yLQmlM6iZfo8fK/file# [following]
--2021-07-09 15:13:47--  https://uc6f05d05a2fd6ac4910e0020da6.dl.dropboxusercontent.com/c

In [3]:
%%capture
!unzip -o participants_data.zip

In [7]:
video_dir = './AlgonautsVideos268_All_30fpsmax'

video_list = glob(video_dir + '/*.mp4')
video_list.sort()

In [2]:
!pip install pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 276kB 7.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44091 sha256=0922afee80885abb24abc78a75488dca66c7bbefb3d291d523846a1204090353
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5MB 23kB/s 
     |████████████████████████████████| 17.3MB 154kB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.0+cu101 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Succes

In [2]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.8")   # please manually install torch 1.8 if Colab changes its default version

1.8.0+cu101 True


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
def visualizedFrames(video, visualizer, predictor):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break

        # Get prediction results for this frame
        outputs = predictor(frame)

        # Make sure the frame is colored
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Draw a visualization of the predictions using the video visualizer
        visualization = visualizer.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

In [5]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
visualizer = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

model_final_f10217.pkl: 178MB [00:10, 16.7MB/s]                           


In [8]:
if not os.path.isdir("./segmentations"):
  os.mkdir("./segmentations")

for vid_path in video_list[:10]:
  video = cv2.VideoCapture(vid_path)
  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frames_per_second = video.get(cv2.CAP_PROP_FPS)
  num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  out_path = "./segmentations/" + vid_path.split("/")[-1][:-4] + "_out.mp4"
  video_writer = cv2.VideoWriter(out_path, fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)
  for visualization in visualizedFrames(video, visualizer, predictor):
    video_writer.write(visualization)
  video.release()
  video_writer.release()
  cv2.destroyAllWindows()
  comp_out_path = out_path[:-4] + "_comp.mp4"
  os.system(f"ffmpeg -i {out_path} -vcodec libx264 {comp_out_path}")
  os.remove(out_path)

In [20]:
from IPython.display import HTML
from base64 import b64encode
import os

segmentations = glob("./segmentations/*.mp4")
s = random.choice(segmentations)
mp4 = open(s,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)